In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [ ]:
col_names = ['Pregnancies', 'Glucose', 'BloodPressure','SkinThickness','Insulin','BMI', 'DiabetesPedigreeFunction','Age', 'Outcome']
data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data.head()

In [ ]:
feature_cols = ['Pregnancies', 'Insulin', 'BMI', 'Age']
X = data[feature_cols]
y = data['Outcome']

In [ ]:
#splitting data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [ ]:
#using logestic regression
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression()
logReg.fit(X_train, y_train)

In [ ]:
y_pred_class  = logReg.predict(X_test)
y_pred_class

# Classification accuracy: percentage of correct predictions

In [ ]:
#calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

# Null Accuracy: accuracy that could be achieved by always predicting the most frequent class

In [ ]:
y_test.value_counts()

In [ ]:
#calculate the % of ones (using mean coz there is only 0 and 1 present)
y_test.mean()

In [ ]:
#calculate the % of 0
1 -  y_test.mean()

In [ ]:
#calculate null accuracy (for binary classification problem coded as 1 and 0)
max(y_test.mean(), 1 -  y_test.mean() )

In [ ]:
#calculate null accuracy (for multi class classification)
y_test.value_counts().head(1)/ len(y_test)

# CONFUSION MATRIX
loosly defined as the table that descricibes the performance of a classification model

In [ ]:
print( metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
confusion = metrics.confusion_matrix(y_test, y_pred_class)
TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]


# Metrics computed from confusion matrix

### classification accuracy: overall how often is the classifier correct?

In [ ]:
#both are equal
print( (TP + TN) / float(TP + TN + FP + FN))
print(metrics.accuracy_score(y_test, y_pred_class))

### Classification error: overall how often is the classifier incorrect

In [ ]:
#Both are equal 
print(( FP+ FN)/ float(TP + TN + FP + FN))
print(1 - metrics.accuracy_score(y_test, y_pred_class))

### sentivity : when the actual value is positive, how often is the prediction correct?

In [ ]:
#Both are same
print(TP/ float( TP +FN))
print( metrics.recall_score(y_test, y_pred_class))

### Specificity : when the actual value is Negative, how often is the prediction correct?

In [ ]:
print(TN/ float(TN+FP))

### False positive rate: when the actual value is Negative, how often is the prediction INcorrect?

In [ ]:
print(FP / float(TN+FP))

### Precision: when the positive value is predicted, how often is the prediction correct 

In [ ]:
print( TP / float(TP+FP))
print(metrics.precision_score(y_test, y_pred_class))

## Adjusting the classification threshold
 

In [ ]:
# print the first 10 predicted responses
logReg.predict(X_test)[0:10]

In [ ]:
# print the first 10 predicted probabilities of class membership
logReg.predict_proba(X_test)[0:10, :]

In [ ]:
# print the first 10 predicted probabilities for class 1
logReg.predict_proba(X_test)[0:10, 1]

In [ ]:
# store the predicted probabilities for class 1
y_pred_prob = logReg.predict_proba(X_test)[:, 1]

In [ ]:
# allow plots to appear in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# histogram of predicted probabilities
plt.hist(y_pred_prob, bins=8)
plt.xlim(0, 1)
plt.title('Histogram of predicted probabilities')
plt.xlabel('Predicted probability of diabetes')
plt.ylabel('Frequency')

In [ ]:
# predict diabetes if the predicted probability is greater than 0.3
from sklearn.preprocessing import binarize
y_pred_class = binarize([y_pred_prob], 0.3)[0]

In [ ]:
# print the first 10 predicted probabilities
y_pred_prob[0:10]

In [ ]:
# print the first 10 predicted classes with the lower threshold
y_pred_class[0:10]

In [ ]:
# previous confusion matrix (default threshold of 0.5)
print(confusion)

In [ ]:
# new confusion matrix (threshold of 0.3)
print(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
# sensitivity has increased (used to be 0.24)
print(46 / float(46 + 16))

In [ ]:
# specificity has decreased (used to be 0.91)
print(80 / float(80 + 50))

**Conclusion:**

- **Threshold of 0.5** is used by default (for binary problems) to convert predicted probabilities into class predictions
- Threshold can be **adjusted** to increase sensitivity or specificity
- Sensitivity and specificity have an **inverse relationship**

## ROC Curves and Area Under the Curve (AUC)

**Question:** Wouldn't it be nice if we could see how sensitivity and specificity are affected by various thresholds, without actually changing the threshold?

**Answer:** Plot the ROC curve!

In [ ]:
# IMPORTANT: first argument is true values, second argument is predicted probabilities
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for diabetes classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

- ROC curve can help you to **choose a threshold** that balances sensitivity and specificity in a way that makes sense for your particular context
- You can't actually **see the thresholds** used to generate the curve on the ROC curve itself

In [ ]:
# define a function that accepts a threshold and prints sensitivity and specificity
def evaluate_threshold(threshold):
    print('Sensitivity:', tpr[thresholds > threshold][-1])
    print('Specificity:', 1 - fpr[thresholds > threshold][-1])

In [ ]:
evaluate_threshold(0.5)

In [ ]:
evaluate_threshold(0.3)

AUC is the **percentage** of the ROC plot that is **underneath the curve**:

In [ ]:
# IMPORTANT: first argument is true values, second argument is predicted probabilities
print(metrics.roc_auc_score(y_test, y_pred_prob))

- AUC is useful as a **single number summary** of classifier performance.
- If you randomly chose one positive and one negative observation, AUC represents the likelihood that your classifier will assign a **higher predicted probability** to the positive observation.
- AUC is useful even when there is **high class imbalance** (unlike classification accuracy).

In [ ]:
# calculate cross-validated AUC
from sklearn.model_selection import cross_val_score
cross_val_score(logReg, X, y, cv=10, scoring='roc_auc').mean()